In [156]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

In [157]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer

In [158]:
!pip install nltk

In [159]:
import string # "string" module is already installed with Python
import nltk
nltk.download('stopwords')
string.punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [160]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jonny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jonny\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jonny\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [161]:
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer


In [162]:
os.listdir(os.path.join(os.getcwd()))

['all_columns.xlsx',
 'data columns.xlsx',
 'omfh_backup.csv',
 'pre_processing.ipynb',
 'pre_processing_lda.ipynb',
 'textual_data.csv']

In [163]:
file_paths = [f for f in os.listdir(os.path.join(os.getcwd())) if f.endswith('.csv')]

file_paths

['omfh_backup.csv', 'textual_data.csv']

In [164]:
dfs = []
for f in file_paths:
    dfs.append(pd.read_csv(os.path.join(os.getcwd(),f), encoding='ISO-8859-1'))
dfs[1]

,What condition(s) do you have that has brought you to this course?
0,Anxiety\r\nEDS - Hypermobility
1,Chronic fatigue\r\nDepression
2,Fibromyalgie
3,NaN
4,Chronic pain / muscular tension
...,...
2634,"Stress of work, trouble to find deep sleep."
2635,"I am a psychotherapist, already using mindfuln..."
2636,Stress and Anxiety
2637,"Work-related and general stress, anxiety and r..."


In [165]:
textual = ['Motivation',
       'Are you able to practice mindfulness exercises/techniques for at least 20 minutes per day for the duration of the course?',
       'Which of these have you done with Breathworks so far? (tick any that apply)',
       'If you have particular access or communication requirements which need to be met in order for you to engage fully with this online retreat, please let us know.',
       'Is someone else paying for your course (not including employer)?',
       'How did you hear about Breathworks?', 'PersonalHistory']
onehote = ['CourseType', 'Gender', 'Ethnicity', 'Is your organisation paying for the course?',
       'City', 'County']
date = ['CourseDate', 'EnrollmentDate',
       'DoB']
columns_to_drop = ['No Label', 'No Label.1', 'If Yes, please provide the name of a person that appears on a receipt']

In [166]:
df = dfs[0].copy().drop(columns=columns_to_drop)

In [167]:
# columns_to_convert = textual
# df = df[columns_to_convert].astype(str)

In [168]:
def clean(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')  # Remove Punctuation
    lowercased = text.lower()  # Lower Case
    tokenized = word_tokenize(lowercased)  # Tokenize
    words_only = [word for word in tokenized if word.isalpha()]  # Remove numbers

    # Make stopword list and add 'yes' to it
    stop_words = set(stopwords.words('english'))
    stop_words.update(['yes','none'])  # Add 'yes' to the set of stopwords

    without_stopwords = [word for word in words_only if not word in stop_words]  # Remove Stop Words
    lemma = WordNetLemmatizer()  # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords]  # Lemmatize
    cleaned = ' '.join(lemmatized)  # Join back to a string
    return cleaned


In [169]:
# for column in textual:
#     df[f'clean_{column}'] = df[column].apply(clean)

# df_c = df.drop(columns=textual)
# df_c.head()

In [170]:
df_test = dfs[1]
df_test.rename(columns={'What condition(s) do you have that has brought you to this course?': 'brought'}, inplace=True)

df_test['brought'] = df_test['brought'].astype(str)


In [171]:
cleaned = df_test['brought'].apply(clean)

In [172]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(cleaned)

lda_model = LatentDirichletAllocation(n_components=3)

lda_vectors = lda_model.fit_transform(data_vectorized)

In [173]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [174]:
print_topics(lda_model, vectorizer)

Topic 0:
[('stress', 340.7284545684226), ('work', 304.35815629000507), ('mindfulness', 272.21237970680636), ('course', 165.8465557750626), ('life', 162.04792453351828), ('like', 154.1891983801961), ('would', 138.98741151275988), ('help', 132.63060502544002), ('year', 114.987446191888), ('practice', 106.94920179967892)]
Topic 1:
[('pain', 551.9993584887495), ('chronic', 326.9561326018829), ('back', 189.9650790546447), ('year', 156.49326750760406), ('anxiety', 137.61635027374703), ('fatigue', 137.31860342279813), ('fibromyalgia', 132.32601779453034), ('syndrome', 113.32545614833009), ('condition', 106.87139123199437), ('depression', 102.64552653444089)]
Topic 2:
[('mindfulness', 881.4401714831554), ('course', 666.7210991771921), ('would', 425.6623510667619), ('year', 360.5192863004976), ('like', 359.42254736501076), ('practice', 352.70817803968816), ('teacher', 288.214786898242), ('meditation', 280.82736128140635), ('stress', 262.22995847005734), ('training', 259.8755874772917)]


In [175]:
example = ["I am looking for more help with my a connection between my mind and body, and maybe for someone to teach me ways to stay more calm and mindfull, i don't suffer from that much physical pain"]

In [176]:
example_vectorized = vectorizer.transform(example)

lda_vectors = lda_model.transform(example_vectorized)

print("topic 0 :", lda_vectors[0][0])
print("topic 1 :", lda_vectors[0][1])
print("topic 1 :", lda_vectors[0][2])

topic 0 : 0.027200785586826714
topic 1 : 0.18504450123726074
topic 1 : 0.7877547131759126


In [177]:
# from sklearn.base import BaseEstimator, TransformerMixin
# class TextPreprocessor(BaseEstimator, TransformerMixin):
#     def __init__(self, func):
#         self.func = func

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         # Ensure X is a DataFrame
#         if isinstance(X, pd.Series):
#             X = X.to_frame()
#         return X.applymap(self.func)


In [178]:
# # Applying the custom text preprocessing function to the textual columns
# text_preprocessing_pipeline = Pipeline(steps=[
#     ('text_preprocessor', TextPreprocessor(preprocessing)),
#     ('tfidf', TfidfVectorizer())
# ])


In [179]:
# preprocessor = ColumnTransformer(transformers=[
#     ('text', text_preprocessing_pipeline, textual),
#     ('onehot', OneHotEncoder(), onehote),
#     # Add date processing as needed
# ], remainder='passthrough')

# # Apply the preprocessing
# # You'll need to convert the output back to a DataFrame if you want to keep using it as such, since ColumnTransformer returns an array
# df_preprocessed = preprocessor.fit_transform(df)